# Params #  
X -> m-by-n matrix of net returns (samples-by-assets)  
r -> m-dimensional vec of net returns of index  
reg -> sparsity regularization parameter  
u -> upper bound of weights  
w_0 -> initial point  
p_neg_exp -> final negative exponent of p  
max_iter -> max number of iterations  

return n-dimensional vector w/ allocation weights on the assets  

In [ ]:
# TODO: Add support for other indices besides S&P (maybe just simple if-else switches)
# TODO: may need to build out a backtest framework that will yield results over longer periods of time
# TODO: Add comments to optimization algorithm along with mathematical formulation (Henry)
# TODO: Fix bugs on tracking algorithm

In [ ]:
import torch
import math
from sec import stock, constants, lookups
from polygon import RESTClient
import yfinance as yf # temporary until we figure out polygon api

In [ ]:
constants.set_polygon_key("_4BtZn3PRCLu6fsdu7dgddb4ucmB1sfp")
poly_cli = RESTClient(api_key="_4BtZn3PRCLu6fsdu7dgddb4ucmB1sfp")
torch.set_default_dtype(torch.float64)

In [ ]:
curr_sp = lookups.get_sp500_tickers()
print(len(curr_sp))
print(curr_sp)

In [ ]:
# aggregate all close data (has been modularized below)
# TODO: remove later after backtesting
close_data = torch.zeros((252, len(curr_sp)))

for i in range(len(curr_sp)):
    aggs = []
    data = poly_cli.list_aggs(ticker=curr_sp[i], multiplier=1, timespan="day", from_="2023-02-01", to = "2024-02-01")
    for j, agg in enumerate(data):
        close_data[j, i] = agg.close

print(close_data)

In [ ]:
torch.set_default_dtype(torch.float64)

col_dim = close_data.shape[0]
return_data = torch.zeros((252, len(curr_sp)))
for i in range(1, col_dim):
    return_data[i] = (close_data[i, :] - close_data[0, :])/close_data[0, :]
print(return_data)

In [ ]:
def getCloseData():
    curr_sp = lookups.get_sp500_tickers()
    close_data = torch.zeros((252, len(curr_sp)))

    for i in range(len(curr_sp)):
        # aggs = []
        data = poly_cli.list_aggs(ticker=curr_sp[i], multiplier=1, timespan="day", from_="2023-02-01", to = "2024-02-01")
        for j, agg in enumerate(data):
            close_data[j, i] = agg.close
    return close_data

In [ ]:
# curr_sp is current s&p tickers
# method simply retrieves close of certain ticker we specify
def getTickerClose(ticker, curr_sp, close_data):
    ix = curr_sp.index(ticker)
    print(close_data[:, ix])

# getTickerClose("AAPL", curr_sp, close_data)

In [ ]:
def calculateTickerReturns():
    col_dim = close_data.shape[0]
    return_data = torch.zeros((252, len(curr_sp)))
    for i in range(1, col_dim):
        return_data[i] = (close_data[i, :] - close_data[0, :])/close_data[0, :]
    return return_data

In [ ]:
def collectSP500():
    index_symbol = "^GSPC"
    index_data = yf.download(index_symbol, period="1y", interval="1d")
    index_close_data = torch.tensor(index_data["Close"].values)
    return index_close_data

In [ ]:
def calculateIndexReturns():
    index_data = collectSP500()
    dim = index_data.shape[0]
    return_data = torch.zeros(252)
    for i in range(1, dim):
        return_data[i] = ((index_data[i] - index_data[0])) / index_data[0]
    return return_data.reshape(252, 1)

In [ ]:
# TODO: currently subscription doesnt have access to index data; will come back
# aggs = []
# for a in poly_cli.list_aggs("I:SPX", 1, "day", "2023-03-10", "2023-05-12", limit=50000):
#     aggs.append(a)

# Algorithm #

In [ ]:
# funcition to solve kkt
def bisection(c, u):
  n = len(c)
  w = torch.zeros(n) # new weight vector
  c_sort, sort_indices = torch.sort(c)
  
  high = n - 1
  low = 0

  while low <= high:
    mid = (low + high) // 2
    if mid == 0:
      break
    mu = -1/mid * (torch.sum(c_sort[0:mid]) + 2)

    cond1 = (mu + c_sort[mid] < 0).item()

    if mid < n:
      cond2 = torch.all((mu + c_sort[mid] >= 0)).item()
    else:
      cond2 = True

    if cond1 and cond2:
      break
    elif cond1 and not cond2:
      low = mid + 1
    else:
      high = mid - 1

  new_values = -(mu + c_sort[:mid] / 2)
  if torch.all(-(mu + c_sort[:mid])/2 <= u).item():
      # print(f"This is w.shape in bisection: {w.shape}")
      # print(f"This is what is getting put in w in bisecttion: {(-(mu + c_sort[:mid])/2).shape}")
      w[sort_indices[:mid]] = -(mu + c_sort[:mid])/2
      # print("We get to the if and this is executed")
      # print(w.shape)
      return w
  else:
    flag = False
    flag2 = False
    k = mid

    while True:
      low1 = 0
      high1 = k - 1

      while low1 <= high1:
        mid1 = (low1 + high1) // 2
        mu = (2 * mid1 * u - torch.sum(c_sort[mid1:k]) - 2) / (k - mid1)

        if mid1 != 0:
          cond1 = torch.all((mu + c_sort[mid1] <= -2*u)).item()
        else:
          cond1 = True

        cond2 = torch.all(((-2 * u) < (mu + c_sort[mid1]))).item() and torch.all(((mu + c_sort[k - 1]) < 0)).item()

        if k < n:
          cond3 = torch.all((mu + c_sort[k]) >= 0).item()
        else:
          cond3 = True

        if cond1 and cond2 and cond3:
          flag = True
          break
        elif cond1 and not cond2:
          low1 = mid1 + 1
        else:
          high1 = mid1 - 1

      if flag:
        break
      else:
        k = k + 1

      if k > n:
        flag2 = True
        break

    if flag2:
      num_elements = int(torch.ceil(torch.tensor(1/u)))
      w[sort_indices[:num_elements]] = u
    else:
      w[sort_indices[:mid1]] = u
      w[sort_indices[mid1:k]] = -(mu + c_sort[mid1:k]) / 2
    return w

In [ ]:
# import torch

# def bisection(c, u):
#     n = len(c)
#     w = torch.zeros(n).reshape(503, 1)
#     c_sort, sort_indices = torch.sort(c)

#     high = n - 1
#     low = 0

#     while low <= high:
#         mid = (low + high) // 2
#         mu = -1/mid * (torch.sum(c_sort[:mid]) + 2)
#         print(mu)

#         tst1 = (mu + c_sort[mid - 1] < 0).item()
#         tst2 = (mu + c_sort[mid] >= 0).item() if mid < n else True

#         if tst1 and tst2:
#             break
#         elif tst1 and not tst2:
#             low = mid + 1
#         else:
#             high = mid - 1

#     if torch.all(-(mu + c_sort[:mid])/2 <= u):
#         print(w)
#         print(-(mu + c_sort[:mid])/2)
#         w[sort_indices[:mid]] = -(mu + c_sort[:mid])/2
#         print("Entered the if loop and this is w")
#         print(w)
#         return w
#     else:
#         flg = False
#         flg2 = False
#         k = mid

#         while True:
#             low1 = 0
#             high1 = k - 1

#             while low1 <= high1:
#                 mid1 = (low1 + high1) // 2
#                 if k - mid1 == 0:
#                     break
#                 mu = (2*mid1*u - torch.sum(c_sort[mid1:k]) - 2) / (k - mid1)

#                 tst1 = (mu + c_sort[mid1 - 1] <= -2*u).item() if mid1 != 0 else True
#                 tst2 = ((-2*u) < (mu + c_sort[mid1])).item() and ((mu + c_sort[k - 1]) < 0).item()

#                 tst3 = (mu + c_sort[k] >= 0).item() if k < n - 1 else True

#                 if tst1 and tst2 and tst3:
#                     flg = True
#                     break
#                 elif tst1 and not tst2:
#                     low1 = mid1 + 1
#                 else:
#                     high1 = mid1 - 1

#             if flg:
#                 break
#             else:
#                 k += 1

#             if k > n:
#                 flg2 = True
#                 break

#         if flg2:
#             num_elements = int(torch.ceil(1/u))
#             w[sort_indices[:num_elements]] = u
#         else:
#             w[sort_indices[:mid1]] = u
#             w[sort_indices[mid1:k]] = -(mu + c_sort[mid1:k])/2

#         return w


In [ ]:
# increasing the weight
def eteMMupdate(w, B, b, Lmax_A, reg, p , c_1, u):
  d = reg / ((p + abs(w)) * c_1)
  c = B @ w + 1/Lmax_A * (b + d) # calculation for q_ete, tracking error, 503 x 503 @ 503 x 1 + 
  
  # print(f"This is c.shape {c.shape}")
  c = c.reshape(503)
  # print(f"This is c after reshape in eteMMupdate {c.shape}")

  return bisection(c,u)

In [ ]:
def track(data, returns, reg, thres = 1e-9, u = 1, w_0 = None, p_neg_exp = 7, max_iter = 1000):
  X = data
  n = X.shape[1] # assets
  m = X.shape[0] # time

  if n == 1:
    print("Must track more than one asset")

  
  w_0 = (torch.ones(n) / n).reshape(503,1) # w_0 ~ Normal
  # print(w_0)
  # print(f"This is {w_0.shape}")

  F_v = torch.zeros((max_iter, 1))

  K = 10
  p_1 = 1 # initial p
  p_k = p_neg_exp # final p
  gamma = (p_k/p_1)**(1/K) # getting deci root
  seq = torch.arange(0, K + 1) # sequential tensor : [0, 1, ... , k]
  exp = gamma**seq # raising gamma to the above sequence (essentially reversing from 1 to p_k) of equiratio intervals
  p_p = p_1 * exp
  p_p = 10**(-p_p) # applying negative base-10 logarithmic transformation, log_10 (p_p)

  p_p_div_10 = p_p/10
  ones = 1e-3 * torch.ones_like(p_p)
  tol = torch.min(p_p_div_10, ones) # tolerance for convergence for early stopping

  k = 0 # iter tracker

  # Using Empirical Tracking Error

  A = 1/m * torch.transpose(X, 0, 1) @ X # building N x N matrix scaled by 1/m

  eigenvalues, _ = torch.linalg.eigh(A) # calculated eigenvalue of A
  Lmax_A = eigenvalues[-1] # retrieving maximum eigenvalue

  B = 2/Lmax_A * (A - Lmax_A * torch.eye(n)) # scaled eigenvector corresponding to Lmax_A
  b = -2/m * torch.transpose(X, 0, 1) @ returns # N x M @ M x 1 = N x 1

  # => Bw_(k) + reg * d_(p,u) + b

  for i in range(1,K+1):
    p = p_p[i]
    c_1 = torch.log(1 + u/p)
    flag = True
    while True:
      k = k + 1
      if k >= max_iter - 1:
        break

      # accelerated scheme for faster convergence (taking a double step)
      w_1 = eteMMupdate(w_0, B, b, Lmax_A, reg, p, c_1, u)
      w_1 = w_1.reshape(503, 1)
      w_2 = eteMMupdate(w_1, B, b, Lmax_A, reg, p, c_1, u)
      w_2 = w_2.reshape(503, 1)
      R = w_1 - w_0
      U = w_2 - w_1 - R
      norm_R = torch.norm(R, p=2)
      norm_U = torch.norm(U, p=2)

      a = max(min(-norm_R / norm_U, -1), -300)

      while True:
        if abs(a + 1) < 1e-6:
          w = w_2
          F_v[k - 1] = 1/reg * torch.norm(torch.matmul(X, w) - returns)**2 + m/c_1 * torch.sum(torch.log(1 + w/p))
          w_0 = w
          break

        w = w_0 - 2 * a * R + a**2 * U
        w = w.reshape(503)
        # print(f"This is w before the bisection: {w}")
        w = w.reshape(503, 1)
        w = bisection(-2 * w, u)
        w = w.reshape(503)
        # print(f"This is w after the bisection: {w}")
        w = w.reshape(503, 1)
        F_v[k - 1] = 1/reg * torch.norm(X @ w - returns)**2 + m/c_1 * torch.sum(torch.log(1 + w/p))

        if flag == 0 and F_v[k - 1] * (1 - torch.sign(F_v[k - 1]) * 1e-9) >= F_v[max(k - 2, 0)]:
          a = (a - 1) / 2
        else:
          w_0 = w
          break

      if flag == 0:
        rel_change = torch.abs(F_v[k - 1] - F_v[k - 2]) / max(1, abs(F_v[k - 1]))

        if rel_change <= tol[i] or k >= max_iter - 1:
          break
      flag = 0
      
  w[w < thres] = 0
  w = w / sum(w)
  return w.reshape(503)


# Testing #

In [ ]:
ticker_returns = calculateTickerReturns()
index_returns = calculateIndexReturns()

print(ticker_returns[0:5])
print(index_returns[0:5])

In [ ]:
# bisection algorithm issues (tensor containing more than one value and being ambiguous)
portfolio_weighting = track(ticker_returns, index_returns, reg=0.2)

In [ ]:
tickers = []
for i in range(len(portfolio_weighting)):
    if portfolio_weighting[i] > 0:
        tickers.append(curr_sp[i])
tickers